In [1]:
# import libs
import pandas as pd
import glob
import os
import re
%matplotlib inline

In [2]:
# define global variables
elec_folder_path = '/users/ianmyjer/desktop/disagg/elec/'
pre_combined_elec_path = '/users/ianmyjer/desktop/disagg/pre_combined/Electricity_P.tab'
weather_hourly_path = '/users/ianmyjer/desktop/disagg/weather/Climate_HourlyWeather.tab'

In [3]:
# Define function to import all files from folder and combine
# based on datetime index (axis = 1)
def tsvs_from_folder(folder_path,meter_type='Electricity'):
    # get all filepaths in folder
    allfiles = glob.glob(folder_path+"/*.tab")
    
    # create a list with the name of each submeter
    name = [re.search('{}_(.*).tab$'.format(meter_type),fp).group(1) for fp in allfiles]

    # provided the file name and corresponding path to each file
    # read in file and parse datetime
    # store in dictionary of dataframes
    datadct = {}
    for k, v in zip(name,allfiles):
        print(k)
        datadct[k] = pd.read_csv(v,delimiter='\t',index_col=0,header=0)
        datadct[k].index = pd.to_datetime(datadct[k].index,unit='s')

    # join everything together based on index        
    df = pd.concat(datadct,axis=1,join='outer')
    df.index.rename('ts',inplace=True)

    return df

In [4]:
# import elec files using function
elec = tsvs_from_folder(elec_folder_path)

B1E


/Users/ianmyjer/anaconda/lib/python3.5/site-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


B2E
BME
CDE
CWE
DNE
DWE
EBE
EQE
FGE
FRE
GRE
HPE
HTE
OFE
OUE
RSE
TVE
UTE
WHE
WOE


In [5]:
# take only the "power" columns
df = elec.xs('P',level=1,axis=1)

In [6]:
# add extra columns based on documentation
df['MHE'] = df['WHE'] - df['RSE'] - df['GRE']
df['UNE'] = df['MHE'] - df.drop(['MHE','WHE','RSE','GRE'],axis=1).sum(axis=1)

/Users/ianmyjer/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/ianmyjer/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [7]:
# turns out this dataset also has pre_combined data. oops
# power = pd.read_csv(pre_combined_elec_path,delimiter='\t',header=0,index_col=0)
# power.index = pd.to_datetime(power.index,unit='s')
# power.index.name = 'ts'

In [8]:
# Rename columns
actual_names = {'B1E':'north_br',
                'B2E':'south_br',
                'BME':'basement',
                'CDE':'dryer',
                'CWE':'washer',
                'DNE':'dining_room',
                'DWE':'dishwasher',
                'EBE':'workbench',
                'EQE':'security_system',
                'FGE':'refrigerator',
                'FRE':'furnace_fan',
                'GRE':'garage',
                'HPE':'heat_pump',
                'HTE':'dhw_heater',
                'MHE':'main_house_total',
                'OFE':'office',
                'OUE':'outside_plug',
                'RSE':'rental_suite',
                'TVE':'entertainment',
                'UTE':'utility_room',
                'UNE':'unmetered',
                'WHE':'whole_house_total',
                'WOE':'oven'}
df.rename(columns = actual_names,inplace=True)

/Users/ianmyjer/anaconda/lib/python3.5/site-packages/pandas/core/frame.py:2844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [9]:
# import weather data
weather = pd.read_csv(weather_hourly_path, 
                      index_col=0, 
                      delimiter='\t', 
                      header=0, 
                      parse_dates=True,
                      usecols=['Date/Time','Temp (C)',
                                'Dew Point Temp (C)',
                                'Rel Hum (%)',
                                'Wind Spd (km/h)',
                                'Stn Press (kPa)'])
weather.index.name = 'ts'

In [10]:
weather.isnull().sum()

Temp (C)              22
Dew Point Temp (C)    36
Rel Hum (%)           38
Wind Spd (km/h)       31
Stn Press (kPa)       35
dtype: int64

In [11]:
# Fill N/A values using pandas interpolate
weather.interpolate(method='time',axis=0,inplace=True)

In [12]:
weather.isnull().sum()

Temp (C)              0
Dew Point Temp (C)    0
Rel Hum (%)           0
Wind Spd (km/h)       0
Stn Press (kPa)       0
dtype: int64

In [13]:
# super stupid code
# but somehow works pretty fast
df['Year'] = df.index.year
df['Month'] = df.index.month
df['Day'] = df.index.day
df['Hour'] = df.index.hour
weather['Year'] = weather.index.year
weather['Month'] = weather.index.month
weather['Day'] = weather.index.day
weather['Hour'] = weather.index.hour
ymdh = ['Year','Month','Day','Hour']
merged = pd.merge(df,weather,on=ymdh,how='left').drop(ymdh,axis=1)
merged.index = df.index
merged.dropna(axis=0,inplace=True)

/Users/ianmyjer/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/ianmyjer/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ianmyjer/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pa

In [14]:
merged.to_csv('/users/ianmyjer/desktop/disagg/electric_data_with_weather.csv')

# Merge Questions

In [15]:
"""
# hella slow merge between two dataframes
# but not super ugly looking
# is there a better way?
weather['per'] = weather.index.to_period('H')
df['per'] = df.index.to_period('H')
merged = pd.merge(df,weather,how='left',on='per').drop('per',axis=1)
merged.index = df.index
merged.dropna(axis=0,inplace=True)
"""

"\n# hella slow merge between two dataframes\n# but not super ugly looking\n# is there a better way?\nweather['per'] = weather.index.to_period('H')\ndf['per'] = df.index.to_period('H')\nmerged = pd.merge(df,weather,how='left',on='per').drop('per',axis=1)\nmerged.index = df.index\nmerged.dropna(axis=0,inplace=True)\n"

In [16]:
'''
Why do none of these merge methods work? 

# key error
a = pd.merge(df,weather,how='left',left_on = df.index.to_period('H'), right_on = weather.index.to_period('H'))

# sort error
a = pd.merge(df.set_index(df.index.to_period('H')), 
             weather.set_index(weather.index.to_period('H')), 
             how = 'left',left_index=True,right_index=True)

# sort error
a = df.set_index(df.index.to_period('H')).merge(weather.set_index(weather.index.to_period('H')), 
             how = 'left',left_index=True,right_index=True)
             
# key error
merged = pd.merge(df, weather, how = 'left',
                  left_on=[df.index.year, df.index.month, df.index.day, df.index.hour], 
                  right_on=[weather.index.year, weather.index.month, weather.index.day, weather.index.hour]
                 ).drop(['key_0','key_1','key_2','key_3'], axis = 1,)
merged.index = df.index
'''

"\nWhy do none of these merge methods work? \n\n# key error\na = pd.merge(df,weather,how='left',left_on = df.index.to_period('H'), right_on = weather.index.to_period('H'))\n\n# sort error\na = pd.merge(df.set_index(df.index.to_period('H')), \n             weather.set_index(weather.index.to_period('H')), \n             how = 'left',left_index=True,right_index=True)\n\n# sort error\na = df.set_index(df.index.to_period('H')).merge(weather.set_index(weather.index.to_period('H')), \n             how = 'left',left_index=True,right_index=True)\n             \n# key error\nmerged = pd.merge(df, weather, how = 'left',\n                  left_on=[df.index.year, df.index.month, df.index.day, df.index.hour], \n                  right_on=[weather.index.year, weather.index.month, weather.index.day, weather.index.hour]\n                 ).drop(['key_0','key_1','key_2','key_3'], axis = 1,)\nmerged.index = df.index\n"